<a href="https://colab.research.google.com/github/JackMcKechnie/Sensitivity-Test-Collection/blob/main/topic_expansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup
!pip install python-terrier
import pyterrier as pt
import pandas as pd
import glob
import os
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import random

if not pt.started():
  pt.init()

! unzip enron_with_categories.zip -d enron_with_categories
! cd enron_with_categories
os.chdir("./enron_with_categories")
os.chdir("./enron_with_categories")

nltk.download('punkt')
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 69 kB 8.1 MB/s 
     |████████████████████████████████| 300 kB 58.7 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 54.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 294 kB 48.8 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
     |████████████████████████████████| 126 kB 52.8 MB/s 
     |████████████████████████████████| 6.4 MB 50.4 MB/s 
     |████████████████████████████████| 287 kB 48.7 MB/s 
     |█████████████████████████

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



Archive:  enron_with_categories.zip
   creating: enron_with_categories/enron_with_categories/
   creating: enron_with_categories/enron_with_categories/1/
 extracting: enron_with_categories/enron_with_categories/1/10425.cats  
  inflating: enron_with_categories/enron_with_categories/1/10425.txt  
 extracting: enron_with_categories/enron_with_categories/1/106296.cats  
  inflating: enron_with_categories/enron_with_categories/1/106296.txt  
  inflating: enron_with_categories/enron_with_categories/1/106298.cats  
  inflating: enron_with_categories/enron_with_categories/1/106298.txt  
  inflating: enron_with_categories/enron_with_categories/1/106588.cats  
  inflating: enron_with_categories/enron_with_categories/1/106588.txt  
 extracting: enron_with_categories/enron_with_categories/1/106590.cats  
  inflating: enron_with_categories/enron_with_categories/1/106590.txt  
 extracting: enron_with_categories/enron_with_categories/1/109359.cats  
  inflating: enron_with_categories/enron_with_cate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Utility functions

def filenames_from_cat(top,second):
    cats_globs = glob.glob('./*/*.cats')
    matches = []
    for g in cats_globs:
        with open(g) as file:
            for line in file:
                line = line.split()[0]
                line = line.split(",")
                if int(line[0]) == top and int(line[1]) == second:
                    dir = os.path.basename(g)
                    filename = os.path.splitext(dir)[0]
                    matches.append(filename)
    return matches

def print_email_from_filename(filename):
    full_filename = glob.glob('./*/*/' + str(filename) + '.txt')
    with open(full_filename[0]) as file:
        for line in file:
            print(line)

def save_email_from_filename(filename):
    full_filename = glob.glob('./*/*' + str(filename) + '.txt')    
    email_contents = ""
    with open(full_filename[0]) as file:
        for line in file:
            email_contents = email_contents + line
    return email_contents

def save_all_from_cat(primary, secondary):
    filenames = filenames_from_cat(primary, secondary)
    contents = ""
    for file in filenames:
        contents = contents + save_email_from_filename(file)
        contents = contents + "\n" + ("*" * 100) + "\n\n"
    return contents

def list_all_filenames():
    globs = glob.glob('./*/*.cats')
    filenames = []
    for file in globs:
        filenames.append(os.path.splitext(os.path.basename(file))[0])
    return filenames

def length_of_email_from_filename(filename):
    email_text = save_email_from_filename(filename)
    tokenised = word_tokenize(email_text)
    return len(tokenised)

def number_of_recipients(filename):
    addresses = []
    contents = save_email_from_filename(filename)
    contents = contents.split("\n")
    for line in contents:
        if line.startswith("To:"):
            addresses.append(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", line))
    
    return len([j for sub in addresses for j in sub])

def get_sender_from_filename(filename):
    addresses = []
    contents = save_email_from_filename(filename)
    contents = contents.split("\n")
    for line in contents:
        if line.startswith("From:"):
            addresses.append(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", line))
    
    return [j for sub in addresses for j in sub][0]

def search_by_id(id):
    all_emails = list_all_filenames()
    for email in all_emails:
        text = save_email_from_filename(email)
        if id in text:
            print(text)

def get_sensitivity(filename):
  if filename in sensitive_filenames:
    return 1
  else:
    return 0

In [3]:
# Preprocessing

stemmer = SnowballStemmer("english")
stop = stopwords.words('english')

def preprocess(email):
  email = word_tokenize(email)
  sentence=""
  for token in email:
    var=stemmer.stem(token)
    if(sentence==""):
      sentence=var
    else:
      sentence=sentence+" "+var
  
  
  sentence = ' '.join([word for word in sentence.split() if word not in (stop)])
  return sentence

In [4]:
# Generate dataframe for collection to be indexed


data = []

# Partition emails into sensitive and non-sensitive
sensitive_filenames = []
# Assume 'Purely personal' and 'Personal but in a professional context' are the sensitive categories
sensitive_filenames.append(filenames_from_cat(1,2))
sensitive_filenames.append(filenames_from_cat(1,3))

# Flatten the list
sensitive_filenames = [j for sub in sensitive_filenames for j in sub]
# Remove duplicates - 3 emails are counted in both categories
sensitive_filenames = list(dict.fromkeys(sensitive_filenames))

non_sensitive_filenames = []
for name in list_all_filenames():
    if name not in sensitive_filenames:
        non_sensitive_filenames.append(name)

for filename in sensitive_filenames:
    email = save_email_from_filename(filename)
    data.append([filename,email,1])

for filename in non_sensitive_filenames:
    email = save_email_from_filename(filename)
    data.append([filename,email,0])

df = pd.DataFrame(data,columns=["docno","text","Sensitivity"])
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
pd_indexer = pt.DFIndexer("/enron_with_categories/enron_with_categories/pd_index")
indexref = pd_indexer.index(df["text"],df["docno"])
index = pt.IndexFactory.of(indexref)

In [ ]:
def print_all_from_query(query):
  # Perform query
  query_result = pt.BatchRetrieve(indexref,wmodel="PL2").search(query)
  # Only positively scored documents considered
  query_result = query_result[query_result['score'] > 0]

  doc_nos = []
  # Top 3 scored documents
  for i in range(len(query_result)):
    if i < 10:
      doc_nos.append(query_result.iloc[i]['docno'])

  # Get how many are sensitive or not
  sensitive_count = 0
  for doc in doc_nos:
    sensitive_count += get_sensitivity(doc)
  
  print(str(sensitive_count) + " / " + str(len(doc_nos)) + " emails sensitive")
  print()

  for doc in doc_nos:
    email = save_email_from_filename(doc)
    print(email)
    print("*"*100)

#print_all_from_query("letter")

In [ ]:
def analyse(query):
  # Perform query
  query_result = pt.BatchRetrieve(indexref,wmodel="PL2").search(query)
  # Only positively scored documents considered
  query_result = query_result[query_result['score'] > 0]

  doc_nos = []
  # Top 3 scored documents
  doc_nos.append(query_result.iloc[0]['docno'])
  doc_nos.append(query_result.iloc[1]['docno'])
  doc_nos.append(query_result.iloc[2]['docno'])

  # Bottom 3 scored documents
  doc_nos.append(query_result.iloc[len(query_result) - 1]['docno'])
  doc_nos.append(query_result.iloc[len(query_result) - 2]['docno'])
  doc_nos.append(query_result.iloc[len(query_result) - 3]['docno'])

  # Get how many are sensitive or not
  sensitive_count = 0
  for doc in doc_nos:
    sensitive_count += get_sensitivity(doc)
  
  print(str(sensitive_count) + " / 6 emails sensitive")
  print()

  for doc in doc_nos:
    email = save_email_from_filename(doc)
    print(email)
    print("*"*100)
  
analyse("letter")

In [ ]:
queries = ["Articles and press releases about US energy policy",
"Editing and approving draft press releases ",
"Legislative action directly affecting Enron ",
"Bush family’s involvement with Enron",
"external business relations",
"recruitment",
"legal proceedings",
"Articles that view the energy industry in a negative light",
"Price caps on Californian energy being voted in and discussed",
"subpoena",
"Dynegy takeover bid",
"protective order",
"px power exchange",
"conference",
"expense reports",
"Lockyer comment",
"agenda",
"North American Electric Reliability Council NERC",
"Public policy contacts in California",
"natural gas prices",
"Confidential information and securities trading",
"Berkley conference",
"California Independent System Operator ISO CISO",
"price mitigation",
"Joe Lieberman",
"Articles",
"Summaries",
"meeting",
"Investment",
"Sacramento",
"Strategic Sourcing GSS",
"Financial mathematics conference",
"Creation and use of a confidential folder to share information",
"Federal Energy Regulatory Commision FERC"]

for query in queries:
  print("Query: " + query.lower())
  #print_all_from_query(query)